In [1]:
pip install cassandra-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 30.8 MB/s eta 0:00:00


In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

def load_config():
    with open("viswan33-token.json") as f:
        secrets = json.load(f)

    cloud_config = {
        'secure_connect_bundle': 'secure-connect-viswan33.zip'
    }

    return cloud_config, secrets["clientId"], secrets["secret"]

def create_connection(cloud_config, client_id, client_secret):
    auth_provider = PlainTextAuthProvider(client_id, client_secret)
    return Cluster(cloud=cloud_config, auth_provider=auth_provider).connect()

def main():
    cloud_config, client_id, client_secret = load_config()
    session = create_connection(cloud_config, client_id, client_secret)

    print('Connected!' if session else "An error occurred.")

if __name__ == "__main__":
    main()


Connected!


In [3]:
import requests
import pandas as pd
from io import StringIO

def fetch_csv(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the status code is not 200
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching file: {e}")
        return None

def convert_csv_to_dataframe(csv_content):
    if csv_content:
        try:
            return pd.read_csv(StringIO(csv_content), on_bad_lines='skip')
        except Exception as e:
            print(f"Error parsing CSV: {e}")
            return None
    return None

def main():
    url = "https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv"
    csv_data = fetch_csv(url)
    df = convert_csv_to_dataframe(csv_data)

    print(df.head() if df is not None else "Failed to load the CSV data.")

if __name__ == "__main__":
    main()


                         Region           Country  Item Type Sales Channel  \
0            Sub-Saharan Africa      South Africa     Fruits       Offline   
1  Middle East and North Africa           Morocco    Clothes        Online   
2         Australia and Oceania  Papua New Guinea       Meat       Offline   
3            Sub-Saharan Africa          Djibouti    Clothes       Offline   
4                        Europe          Slovakia  Beverages       Offline   

  Order Priority  Order Date   Order ID   Ship Date  UnitsSold  UnitPrice  \
0              M   7/27/2012  443368995   7/28/2012       1593       9.33   
1              M   9/14/2013  667593514  10/19/2013       4611     109.28   
2              M   5/15/2015  940995585    6/4/2015        360     421.89   
3              H   5/17/2017  880811536    7/2/2017        562     109.28   
4              L  10/26/2016  174590194   12/4/2016       3973      47.45   

   UnitCost  TotalRevenue  TotalCost  TotalProfit  
0      6.92     

In [4]:
session.execute("""
CREATE TABLE IF NOT EXISTS mydatabase.bronze_sales (
    id UUID PRIMARY KEY,
    transaction_id text,
    customer_id text,
    product_id text,
    amount decimal,
    transaction_date text
)
""")

NameError: name 'session' is not defined

In [5]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

def load_config():
    cloud_config = {
        'secure_connect_bundle': 'secure-connect-viswan33.zip'
    }

    with open("viswan33-token.json") as f:
        secrets = json.load(f)

    client_id = secrets["clientId"]
    client_secret = secrets["secret"]

    return cloud_config, client_id, client_secret


def connect_to_cluster(cloud_config, client_id, client_secret):
    auth_provider = PlainTextAuthProvider(client_id, client_secret)
    cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
    return cluster.connect()

def create_table(session):
    session.execute("""
    CREATE TABLE IF NOT EXISTS mydatabase.bronze_sales (
        id UUID PRIMARY KEY,
        transaction_id text,
        customer_id text,
        product_id text,
        amount decimal,
        transaction_date text
    )
    """)

def main():
    cloud_config, client_id, client_secret = load_config()
    session = connect_to_cluster(cloud_config, client_id, client_secret)

    if session:
        print("Connected to the cluster!")
        create_table(session)
        print("Table created (if it did not exist).")
    else:
        print("An error occurred while connecting to the cluster.")

if __name__ == "__main__":
    main()


Connected to the cluster!
Table created (if it did not exist).


In [6]:
import pandas as pd
from uuid import uuid4
from datetime import datetime
import requests
from io import StringIO
url = "https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv"
response = requests.get(url)
csv_data = response.text

df = pd.read_csv(StringIO(csv_data))
df.columns = df.columns.str.strip().str.replace(' ', '_')

def insert_data_to_cassandra(row):
    try:

        transaction_date = datetime.strptime(row['Order_Date'], '%m/%d/%Y').strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        print(f"Invalid date format: {row['Order_Date']}")
        return None


    session.execute("""
    INSERT INTO mydatabase.bronze_sales (
        id, transaction_id, customer_id, product_id, amount, transaction_date
    ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        uuid4(),
        str(row['Order_ID']),
        str(row['Region']),
        str(row['Item_Type']),
        float(row['TotalRevenue']),
        transaction_date
    ))


df.apply(insert_data_to_cassandra, axis=1)


print(df.columns)


NameError: name 'session' is not defined

In [7]:
import pandas as pd
from uuid import uuid4
from datetime import datetime
import requests
from io import StringIO
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# Load the config to get credentials and cloud configuration
def load_config():
    cloud_config = {
        'secure_connect_bundle': 'secure-connect-viswan33.zip'  # Path to the secure connect bundle
    }

    with open("viswan33-token.json") as f:
        secrets = json.load(f)

    client_id = secrets["clientId"]
    client_secret = secrets["secret"]

    return cloud_config, client_id, client_secret

# Function to connect to Cassandra cluster
def connect_to_cassandra():
    cloud_config, client_id, client_secret = load_config()
    auth_provider = PlainTextAuthProvider(client_id, client_secret)
    cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
    return cluster.connect()

# Fetch CSV data
url = "https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv"
response = requests.get(url)
csv_data = response.text

# Load CSV data into DataFrame
df = pd.read_csv(StringIO(csv_data))
df.columns = df.columns.str.strip().str.replace(' ', '_')

# Function to insert data into Cassandra
def insert_data_to_cassandra(row, session):
    try:
        transaction_date = datetime.strptime(row['Order_Date'], '%m/%d/%Y').strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        print(f"Invalid date format: {row['Order_Date']}")
        return None

    # Insert data into Cassandra table
    session.execute("""
    INSERT INTO mydatabase.bronze_sales (
        id, transaction_id, customer_id, product_id, amount, transaction_date
    ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        uuid4(),
        str(row['Order_ID']),
        str(row['Region']),
        str(row['Item_Type']),
        float(row['TotalRevenue']),
        transaction_date
    ))

# Main function to handle the logic
def main():
    session = connect_to_cassandra()
    if session:
        print("Connected to Cassandra cluster!")
        df.apply(lambda row: insert_data_to_cassandra(row, session), axis=1)
        print("Data insertion completed.")
    else:
        print("Failed to connect to Cassandra cluster.")

if __name__ == "__main__":
    main()



Connected to Cassandra cluster!
Data insertion completed.


In [10]:
from datetime import datetime
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# Load the config to get credentials and cloud configuration
def load_config():
    cloud_config = {
        'secure_connect_bundle': 'secure-connect-viswan33.zip'  # Path to the secure connect bundle
    }

    with open("viswan33-token.json") as f:
        secrets = json.load(f)

    client_id = secrets["clientId"]
    client_secret = secrets["secret"]

    return cloud_config, client_id, client_secret

# Function to connect to Cassandra cluster
def connect_to_cassandra():
    cloud_config, client_id, client_secret = load_config()
    auth_provider = PlainTextAuthProvider(client_id, client_secret)
    cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
    return cluster.connect()

# Transform date format
def transform_date(transaction_date):
    for fmt in ['%Y-%m-%d %H:%M:%S', '%Y-%m-%d']:
        try:
            return datetime.strptime(transaction_date, fmt).date()
        except ValueError:
            continue
    print(f"Invalid date format: {transaction_date}")
    return None

# Create the silver_sales table
def create_silver_table(session):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS mydatabase.silver_sales (
        transaction_id text PRIMARY KEY,
        customer_id text,
        product_id text,
        amount decimal,
        transaction_date date
    )
    """
    session.execute(create_table_query)

# Insert data into silver_sales
def insert_to_silver_sales(session, rows):
    for row in rows:
        transaction_date = transform_date(row.transaction_date)
        if transaction_date:
            session.execute("""
            INSERT INTO mydatabase.silver_sales (transaction_id, customer_id, product_id, amount, transaction_date)
            VALUES (%s, %s, %s, %s, %s)
            """, (
                row.transaction_id,
                row.customer_id,
                row.product_id,
                row.amount,
                transaction_date
            ))

# Main function
def main():
    session = connect_to_cassandra()  # Initialize the session
    if session:
        print("Connected to Cassandra cluster!")
        create_silver_table(session)

        # Fetch rows from the bronze_sales table
        query = "SELECT transaction_id, customer_id, product_id, amount, transaction_date FROM mydatabase.bronze_sales"
        rows = session.execute(query)

        # Insert rows into the silver_sales table
        insert_to_silver_sales(session, rows)
    else:
        print("Failed to connect to Cassandra cluster.")

if __name__ == "__main__":
    main()


Connected to Cassandra cluster!


In [15]:
from collections import defaultdict
from decimal import Decimal
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# Load the configuration for Cassandra credentials and cloud info
def load_config():
    cloud_config = {
        'secure_connect_bundle': 'secure-connect-viswan33.zip'  # Path to the secure connect bundle
    }

    with open("viswan33-token.json") as f:
        secrets = json.load(f)

    client_id = secrets["clientId"]
    client_secret = secrets["secret"]

    return cloud_config, client_id, client_secret

# Function to connect to the Cassandra cluster and return a session
def connect_to_cassandra():
    cloud_config, client_id, client_secret = load_config()
    auth_provider = PlainTextAuthProvider(client_id, client_secret)
    cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
    return cluster.connect()

# Function to execute any query on Cassandra
def execute_query(session, query, params=None):
    if params:
        session.execute(query, params)
    else:
        session.execute(query)

# Function to aggregate sales by any group column
def aggregate_sales_data(session, query, group_by_column):
    rows = session.execute(query)
    aggregated_data = defaultdict(float)

    for row in rows:
        # Using the group_by_column dynamically with getattr
        group_value = getattr(row, group_by_column)
        aggregated_data[group_value] += float(row.amount)

    return aggregated_data

# Function to insert aggregated data into a specific table
def insert_aggregated_data(session, table, data, group_column, value_column):
    for key, value in data.items():
        query = f"""
        INSERT INTO {table} ({group_column}, {value_column})
        VALUES (%s, %s)
        """
        execute_query(session, query, (key, value))

# Create table helper function with dynamic table structure
def create_table_if_not_exists(session, table_name, columns):
    query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} ({columns})
    """
    execute_query(session, query)

# Main function that orchestrates the operations
def main():
    # Establish a connection to Cassandra and get the session
    session = connect_to_cassandra()

    if session:
        print("Connected to Cassandra cluster!")

        # Create tables for gold_sales_by_customer, gold_sales_by_product, and gold_daily_sales
        create_table_if_not_exists(session,
            "mydatabase.gold_sales_by_customer",
            "customer_id text PRIMARY KEY, total_sales decimal"
        )

        create_table_if_not_exists(session,
            "mydatabase.gold_sales_by_product",
            "product_id text PRIMARY KEY, total_sales decimal"
        )

        create_table_if_not_exists(session,
            "mydatabase.gold_daily_sales",
            "transaction_date date PRIMARY KEY, total_sales decimal"
        )

        # Aggregate sales by customer, product, and daily
        customer_sales_query = "SELECT customer_id, amount FROM mydatabase.silver_sales"
        customer_sales = aggregate_sales_data(session, customer_sales_query, 'customer_id')
        insert_aggregated_data(session, "mydatabase.gold_sales_by_customer", customer_sales, "customer_id", "total_sales")

        product_sales_query = "SELECT product_id, amount FROM mydatabase.silver_sales"
        product_sales = aggregate_sales_data(session, product_sales_query, 'product_id')
        insert_aggregated_data(session, "mydatabase.gold_sales_by_product", product_sales, "product_id", "total_sales")

        daily_sales_query = "SELECT transaction_date, amount FROM mydatabase.silver_sales"
        daily_sales = aggregate_sales_data(session, daily_sales_query, 'transaction_date')
        insert_aggregated_data(session, "mydatabase.gold_daily_sales", daily_sales, "transaction_date", "total_sales")

    else:
        print("Failed to connect to Cassandra cluster.")

if __name__ == "__main__":
    main()


Connected to Cassandra cluster!


In [18]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# Load the configuration for Cassandra credentials and cloud info
def load_config():
    cloud_config = {
        'secure_connect_bundle': 'secure-connect-viswan33.zip'  # Path to the secure connect bundle
    }

    with open("viswan33-token.json") as f:
        secrets = json.load(f)

    client_id = secrets["clientId"]
    client_secret = secrets["secret"]

    return cloud_config, client_id, client_secret

# Function to connect to the Cassandra cluster and return a session
def connect_to_cassandra():
    cloud_config, client_id, client_secret = load_config()
    auth_provider = PlainTextAuthProvider(client_id, client_secret)
    cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
    return cluster.connect()

# Function to execute any query on Cassandra
def execute_query(session, query):
    return session.execute(query)

# Main function to fetch and print data from Gold tables
def main():
    # Establish a connection to Cassandra and get the session
    session = connect_to_cassandra()

    if session:
        print("Connected to Cassandra cluster!")

        # Fetch data from Gold Table 1: Total Sales by Customer
        result_customer = execute_query(session, "SELECT * FROM mydatabase.gold_sales_by_customer")
        print("Sales by Customer:")
        for row in result_customer:
            print(row)

        # Fetch data from Gold Table 2: Total Sales by Product
        result_product = execute_query(session, "SELECT * FROM mydatabase.gold_sales_by_product")
        print("Sales by Product:")
        for row in result_product:
            print(row)

        # Fetch data from Gold Table 3: Total Sales by Day
        result_daily = execute_query(session, "SELECT * FROM mydatabase.gold_daily_sales")
        print("Sales by Day:")
        for row in result_daily:
            print(row)
    else:
        print("Failed to connect to Cassandra cluster.")

if __name__ == "__main__":
    main()


Connected to Cassandra cluster!
Sales by Customer:
Row(customer_id='Australia and Oceania', total_sales=Decimal('10711258.13'))
Row(customer_id='Europe', total_sales=Decimal('34964749.830000006'))
Row(customer_id='Middle East and North Africa', total_sales=Decimal('24765127.25'))
Row(customer_id='Central America and the Caribbean', total_sales=Decimal('17570835.42'))
Row(customer_id='Asia', total_sales=Decimal('28840812.190000005'))
Row(customer_id='Sub-Saharan Africa', total_sales=Decimal('24225437.419999998'))
Row(customer_id='North America', total_sales=Decimal('3611757.5199999996'))
Sales by Product:
Row(product_id='Household', total_sales=Decimal('38519082.8'))
Row(product_id='Office Supplies', total_sales=Decimal('27880904.94'))
Row(product_id='Vegetables', total_sales=Decimal('1135114.08'))
Row(product_id='Snacks', total_sales=Decimal('2193642.66'))
Row(product_id='Personal Care', total_sales=Decimal('3191147.8499999996'))
Row(product_id='Meat', total_sales=Decimal('21278865.93'